In [ ]:
#Khai báo thư viện
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, when, isnull
from pyspark.sql import functions as F
from pyspark.sql.functions import desc, asc
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
#Đọc file csv được thu thập từ trang https://finance.yahoo.com/
spark = SparkSession.builder.getOrCreate()
data = spark.read.csv("NVDA.csv", header=True, inferSchema=True)
data = data.drop("Adj Close")

In [ ]:
data.toPandas()

,Date,Open,High,Low,Close,Volume
0,1999-01-22,0.437500,0.488281,0.388021,0.410156,271468800
1,1999-01-25,0.442708,0.458333,0.410156,0.453125,51048000
2,1999-01-26,0.458333,0.467448,0.411458,0.417969,34320000
3,1999-01-27,0.419271,0.429688,0.395833,0.416667,24436800
4,1999-01-28,0.416667,0.419271,0.412760,0.415365,22752000
...,...,...,...,...,...,...
6274,2023-12-28,496.429993,498.839996,494.119995,495.220001,24658700
6275,2023-12-29,498.130005,499.970001,487.510010,495.220001,38869000
6276,2024-01-02,492.440002,492.950012,475.950012,481.679993,41125400
6277,2024-01-03,474.850006,481.839996,473.200012,475.690002,32089600


In [ ]:
actual = data.orderBy(desc('Date')).limit(30)
actual = actual.orderBy('Date', ascending=True)

In [ ]:
actual.show(30)

+----------+------------------+------------------+------------------+------------------+--------+
|      Date|              Open|              High|               Low|             Close|  Volume|
+----------+------------------+------------------+------------------+------------------+--------+
|2023-11-21|  501.260009765625| 505.1700134277344| 492.2200012207031|499.44000244140625|56574700|
|2023-11-22| 498.5199890136719| 503.3500061035156| 476.8999938964844| 487.1600036621094|89942000|
|2023-11-24|484.70001220703125| 489.2099914550781|477.45001220703125|  477.760009765625|29464500|
|2023-11-27|             478.0|485.29998779296875| 476.5199890136719| 482.4200134277344|39566200|
|2023-11-28| 482.3599853515625| 483.2300109863281| 474.7300109863281| 478.2099914550781|40149100|
|2023-11-29| 483.7900085449219| 487.6199951171875| 478.6000061035156| 481.3999938964844|38200500|
|2023-11-30|  480.239990234375| 481.1000061035156| 464.2200012207031|467.70001220703125|52624700|
|2023-12-01|        

In [ ]:
data = data.subtract(actual)
data = data.orderBy('Date', ascending=True)

In [ ]:
data.toPandas()

,Date,Open,High,Low,Close,Volume
0,1999-01-22,0.437500,0.488281,0.388021,0.410156,271468800
1,1999-01-25,0.442708,0.458333,0.410156,0.453125,51048000
2,1999-01-26,0.458333,0.467448,0.411458,0.417969,34320000
3,1999-01-27,0.419271,0.429688,0.395833,0.416667,24436800
4,1999-01-28,0.416667,0.419271,0.412760,0.415365,22752000
...,...,...,...,...,...,...
6244,2023-11-14,496.799988,498.339996,490.399994,496.559998,41695400
6245,2023-11-15,499.350006,499.600006,482.000000,488.880005,47549700
6246,2023-11-16,486.790009,495.250000,483.299988,494.799988,33975600
6247,2023-11-17,495.239990,497.170013,490.070007,492.980011,32520500


In [ ]:
#Tạo cột giá đóng sau 30 ngày tương ứng với từng ngày.
forecast_out =30
window_spec = Window().orderBy("Date")
data_30 = data.withColumn("Close_after_30_days", F.lead("Close", forecast_out).over(window_spec))

In [ ]:
data_30.show()

+----------+-------------------+-------------------+-------------------+-------------------+---------+-------------------+
|      Date|               Open|               High|                Low|              Close|   Volume|Close_after_30_days|
+----------+-------------------+-------------------+-------------------+-------------------+---------+-------------------+
|1999-01-22|             0.4375| 0.4882810115814209|0.38802099227905273| 0.4101560115814209|271468800| 0.4453130066394806|
|1999-01-25|0.44270798563957214|0.45833298563957214| 0.4101560115814209|           0.453125| 51048000| 0.4596349895000458|
|1999-01-26|0.45833298563957214|0.46744799613952637|0.41145798563957214| 0.4179689884185791| 34320000|0.45182299613952637|
|1999-01-27|0.41927099227905273| 0.4296880066394806|0.39583298563957214|0.41666701436042786| 24436800|0.43229201436042786|
|1999-01-28|0.41666701436042786|0.41927099227905273| 0.4127599895000458| 0.4153650104999542| 22752000|0.42447900772094727|
|1999-01-29| 0.4

In [ ]:
#Trong df mới thì 30 dòng cuối không có giá đóng sau 30 ngày nên dùng làm tập dự đoán (streaming)
df = data_30.toPandas()
df.tail(40)

,Date,Open,High,Low,Close,Volume,Close_after_30_days
6209,2023-09-26,420.010010,428.200012,416.549988,419.109985,40228200,459.549988
6210,2023-09-27,423.299988,428.720001,416.290009,424.679993,44493500,465.739990
6211,2023-09-28,424.600006,434.459991,421.149994,430.890015,42466300,469.500000
6212,2023-09-29,438.269989,441.440002,433.070007,434.989990,39722100,483.350006
6213,2023-10-02,440.299988,451.750000,438.609985,447.820007,43329800,486.200012
6214,2023-10-03,448.079987,451.299988,432.459991,435.170013,47085000,496.559998
6215,2023-10-04,437.420013,441.429993,432.920013,440.410004,36182100,488.880005
6216,2023-10-05,440.500000,449.000000,438.880005,446.880005,39348300,494.799988
6217,2023-10-06,441.929993,457.890015,440.260010,457.619995,43339700,492.980011
6218,2023-10-09,448.420013,456.049988,443.679993,452.730011,40967500,504.089996


In [ ]:
stream = data_30.filter(isnull("Close_after_30_days"))
stream.show(30, truncate=False)

+----------+------------------+------------------+------------------+------------------+--------+-------------------+
|Date      |Open              |High              |Low               |Close             |Volume  |Close_after_30_days|
+----------+------------------+------------------+------------------+------------------+--------+-------------------+
|2023-10-10|453.1000061035156 |462.5899963378906 |450.8800048828125 |457.9800109863281 |36858200|NULL               |
|2023-10-11|461.9599914550781 |468.5899963378906 |460.5             |468.05999755859375|37813700|NULL               |
|2023-10-12|467.7699890136719 |476.0899963378906 |463.29998779296875|469.45001220703125|48132500|NULL               |
|2023-10-13|469.6000061035156 |471.1600036621094 |452.79998779296875|454.6099853515625 |47411500|NULL               |
|2023-10-16|450.6300048828125 |462.25            |449.1199951171875 |460.95001220703125|37509900|NULL               |
|2023-10-17|440.0             |447.5400085449219 |424.79

In [ ]:
train = data_30.subtract(stream)

In [ ]:
train.toPandas()

,Date,Open,High,Low,Close,Volume,Close_after_30_days
0,1999-01-22,0.437500,0.488281,0.388021,0.410156,271468800,0.445313
1,1999-01-25,0.442708,0.458333,0.410156,0.453125,51048000,0.459635
2,1999-01-26,0.458333,0.467448,0.411458,0.417969,34320000,0.451823
3,1999-01-27,0.419271,0.429688,0.395833,0.416667,24436800,0.432292
4,1999-01-28,0.416667,0.419271,0.412760,0.415365,22752000,0.424479
...,...,...,...,...,...,...,...
6214,2023-10-03,448.079987,451.299988,432.459991,435.170013,47085000,496.559998
6215,2023-10-04,437.420013,441.429993,432.920013,440.410004,36182100,488.880005
6216,2023-10-05,440.500000,449.000000,438.880005,446.880005,39348300,494.799988
6217,2023-10-06,441.929993,457.890015,440.260010,457.619995,43339700,492.980011


In [ ]:
actual_pd = actual.toPandas()
actual_pd.to_csv("actual.csv", index=False)

In [ ]:
stream_pd = stream.toPandas()
stream_pd.to_csv("stream.csv", index=False)

In [ ]:
train_pd = train.toPandas()
train_pd.to_csv("train.csv", index=False)

Dự đoán